# 국민건강영양조사 데이터를 이용하여 우울감 예측 머신러닝 알고리즘 개발

파이썬 주피터 화면 조정

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

필요한 라이브러리 임포트

In [1]:
from torch.utils.data import TensorDataset
import numpy as np
from numpy import concatenate
from numpy import loadtxt
import pandas as pd
from pandas import read_csv,DataFrame,concat
from pandas.plotting import register_matplotlib_converters
from math import sqrt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,random_split,DistributedSampler,DataLoader

import random,os,copy
import pickle
from tqdm import tqdm
from collections import Counter

from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error,r2_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder

import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import pyplot
from matplotlib import rc
import seaborn as sns

현재 디렉토리 체크

In [2]:
os.getcwd()

'C:\\Users\\pc'

훈련 및 테스트에 필요한 데이터 불러오기

In [3]:
df1=pd.read_sas("C:/Users/pc/Desktop/~/hn14_all.sas7bdat", encoding="ISO-8859-1")
df2=pd.read_sas("C:/Users/pc/Desktop/~/hn16_all.sas7bdat", encoding="ISO-8859-1")
df3=pd.read_sas("C:/Users/pc/Desktop/~/hn18_all.sas7bdat", encoding="ISO-8859-1")
df4=pd.read_sas("C:/Users/pc/Desktop/~/hn20_all.sas7bdat", encoding="ISO-8859-1")

In [4]:
pd.set_option('display.max_seq_items', None)

등록되어 있는 데이터셋의 칼럼 이름

In [5]:
df1.columns

Index(['mod_d', 'id', 'ID_fam', 'year', 'region', 'town_t', 'apt_t', 'psu',
       'sex', 'age', 'age_month', 'incm', 'ho_incm', 'incm5', 'ho_incm5',
       'edu', 'occp', 'wt_hs', 'wt_itvex', 'wt_pft', 'wt_tr', 'wt_ntr',
       'wt_tot', 'wt_pftr', 'wt_pfnt', 'wt_trnt', 'wt_pftrnt', 'kstrata',
       'cfam', 'genertn', 'allownc', 'house', 'live_t', 'ainc_unit1', 'ainc_1',
       'ainc', 'marri_1', 'marri_2', 'fam_rela', 'tins', 'npins', 'id_f',
       'id_m', 'D_1_1', 'D_2_1', 'D_2_wk', 'DI1_dg', 'DI1_ag', 'DI1_pr',
       'DI1_pt', 'DI1_2', 'DI2_dg', 'DI2_ag', 'DI2_pr', 'DI2_pt', 'DI2_2',
       'DI3_dg', 'DI3_ag', 'DI3_pr', 'DI3_pt', 'DI3_2', 'DI4_dg', 'DI4_pr',
       'DI4_pt', 'DI5_dg', 'DI5_ag', 'DI5_pr', 'DI5_pt', 'DI6_dg', 'DI6_ag',
       'DI6_pr', 'DI6_pt', 'DM1_dg', 'DM1_pr', 'DM1_pt', 'DM2_dg', 'DM2_ag',
       'DM2_pr', 'DM2_pt', 'DM3_dg', 'DM3_ag', 'DM3_pr', 'DM3_pt', 'D_8_1',
       'D_8_1_s', 'D_8_2', 'D_8_2_t', 'D_8_3', 'D_8_3_s', 'D_8_4', 'DJ2_dg',
       'DJ2_ag', 'D

학습에 필요한 변수 추출

In [6]:
df1a=df1[['sex','age','region','incm','edu','occp','genertn','allownc','ainc','marri_1','DI1_dg','DI2_dg','DI3_dg',
         'DI5_dg','DI6_dg','DM2_dg','DM3_dg','DJ2_dg','DE2_dg','DE1_dg','DC1_dg','DC2_dg','DC3_dg','DC4_dg',
         'DC5_dg','DC6_dg','DC7_dg','DF2_dg','DF2_pr','DF2_pt','DL1_dg','DJ8_dg','LQ_1EQL', 
         'LQ_2EQL','LQ_3EQL','LQ_4EQL','LQ_5EQL','EQ5D','BO1_1','BD1','mh_PHQ_S','BP1','BP6_10','BP6_2','BP6_31',
         'BS1_1','sm_presnt','BE3_85','BE3_86','BE3_75','BE3_76','HE_BMI','HE_glu','HE_chol','HE_TG']]

df2a=df2[['sex','age','region','incm','edu','occp','genertn','allownc','ainc','marri_1','DI1_dg','DI2_dg','DI3_dg',
         'DI5_dg','DI6_dg','DM2_dg','DM3_dg','DJ2_dg','DE2_dg','DE1_dg','DC1_dg','DC2_dg','DC3_dg','DC4_dg',
         'DC5_dg','DC6_dg','DC7_dg','DF2_dg','DF2_pr','DF2_pt','DL1_dg','DJ8_dg','LQ_1EQL', 
         'LQ_2EQL','LQ_3EQL','LQ_4EQL','LQ_5EQL','EQ5D','BO1_1','BD1','mh_PHQ_S','BP1','BP6_10','BP6_2','BP6_31',
         'BS1_1','sm_presnt','BE3_85','BE3_86','BE3_75','BE3_76','HE_BMI','HE_glu','HE_chol','HE_TG']]

df3a=df3[['sex','age','region','incm','edu','occp','genertn','allownc','ainc','marri_1','DI1_dg','DI2_dg','DI3_dg',
         'DI5_dg','DI6_dg','DM2_dg','DM3_dg','DJ2_dg','DE2_dg','DE1_dg','DC1_dg','DC2_dg','DC3_dg','DC4_dg',
         'DC5_dg','DC6_dg','DC7_dg','DF2_dg','DF2_pr','DF2_pt','DL1_dg','DJ8_dg','LQ_1EQL', 
         'LQ_2EQL','LQ_3EQL','LQ_4EQL','LQ_5EQL','EQ5D','BO1_1','BD1','mh_PHQ_S','BP1','BP6_10','BP6_2','BP6_31',
         'BS1_1','sm_presnt','BE3_85','BE3_86','BE3_75','BE3_76','HE_BMI','HE_glu','HE_chol','HE_TG']]

df4a=df4[['sex','age','region','incm','edu','occp','genertn','allownc','ainc','marri_1','DI1_dg','DI2_dg','DI3_dg',
         'DI5_dg','DI6_dg','DM2_dg','DM3_dg','DJ2_dg','DE2_dg','DE1_dg','DC1_dg','DC2_dg','DC3_dg','DC4_dg',
         'DC5_dg','DC6_dg','DC7_dg','DF2_dg','DF2_pr','DF2_pt','DL1_dg','DJ8_dg','LQ_1EQL', 
         'LQ_2EQL','LQ_3EQL','LQ_4EQL','LQ_5EQL','EQ5D','BO1_1','BD1','mh_PHQ_S','BP1','BP6_10','BP6_2','BP6_31',
         'BS1_1','sm_presnt','BE3_85','BE3_86','BE3_75','BE3_76','HE_BMI','HE_glu','HE_chol','HE_TG']]

결측치 제거 : null 값

In [7]:
df1a=df1a.dropna(axis=0)

In [8]:
df2a=df2a.dropna(axis=0)

In [9]:
df3a=df3a.dropna(axis=0)

In [10]:
df4a=df4a.dropna(axis=0)

In [11]:
df1a.isnull().sum()

sex          0
age          0
region       0
incm         0
edu          0
occp         0
genertn      0
allownc      0
ainc         0
marri_1      0
DI1_dg       0
DI2_dg       0
DI3_dg       0
DI5_dg       0
DI6_dg       0
DM2_dg       0
DM3_dg       0
DJ2_dg       0
DE2_dg       0
DE1_dg       0
DC1_dg       0
DC2_dg       0
DC3_dg       0
DC4_dg       0
DC5_dg       0
DC6_dg       0
DC7_dg       0
DF2_dg       0
DF2_pr       0
DF2_pt       0
DL1_dg       0
DJ8_dg       0
LQ_1EQL      0
LQ_2EQL      0
LQ_3EQL      0
LQ_4EQL      0
LQ_5EQL      0
EQ5D         0
BO1_1        0
BD1          0
mh_PHQ_S     0
BP1          0
BP6_10       0
BP6_2        0
BP6_31       0
BS1_1        0
sm_presnt    0
BE3_85       0
BE3_86       0
BE3_75       0
BE3_76       0
HE_BMI       0
HE_glu       0
HE_chol      0
HE_TG        0
dtype: int64

### 변수 설명
#### (1) sex (성별) : 1.남자, 2.여자
#### (2) age (만나이): xx세
#### (3) region (17개시도): 1.서울, 2.부산, 3.대구, 4.인천, 5.광주, 6.대전, 7.울산, 8.세종, 9.경기, 10.강원, 11.충북, 12.충남, 13.전북, 14.전남, 15.경북, 16.경남, 17.제주
#### (4) incm (개인 소득 4분위수): 1.하, 2.중하, 3.중상, 4.상
#### (5) edu (교육수준 재분류 코드): 1.초졸이하, 2.중졸, 3.고졸, 4.대졸이상
#### (6) occp (직업재분류 및 실업/비경제활동): 1 관리자, 전문가 및 관련 종사자, 2.사무종사자, 3.서비스 및 판매 종사자, 4.농림어업 숙련 종사자, 5.기능원, 장치․기계조작 및 조립종사자, 6.단순노무종사자, 7.무직(주부, 학생 등)
#### (7) genertn (가구 세대구성코드): 1.1세대 가구 - 1인가구, 2.1세대 가구 - 부부, 3.1세대 가구 - 기타, 4.2세대 가구 - 부부+미혼자녀, 5.2세대 가구 – 편부모+미혼자녀, 6.2세대 가구 – 기타, 7.3세대 이상 가구, 9.모름, 무응답
#### (8) allownc (기초생활수급 여부): 10.수급경험 있음(과거 또는 현재), 20.수급경험 없음, 99.모름, 무응답
#### (9) ainc (월평균 가구총소득): xx만원, (17.월 17만원 이하, xx만원.17초과 ~ 1500미만 (연속형 소득금액), 1500.월 1,500만원 이상)
#### (10) marri_1 (결혼여부): 1.기혼, 2.미혼, 9.모름, 무응답
#### (11) DI1_dg (고혈압 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (12) DI2_ag (이상지질혈증 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (13) DI3_dg (뇌졸중 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (14) DI5_dg (심근경색증 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (15) DI6_dg (협심증 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (16) DM2_dg (골관절염 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (17) DM3_dg (류마티스성 관절염 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (18) DJ2_dg (폐결핵 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (19) DE2_dg (갑상선 질환 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (20) DE1_dg (당뇨병 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (21) DC1_dg (위암 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (22) DC2_dg (간암 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (23) DC3_dg (대장암 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (24) DC4_dg (유방암 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (25) DC5_dg (자궁경부암 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (26) DC6_dg (폐암 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (27) DC7_dg (갑상선암 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (28) DF2_dg (우울증 의사진단 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (29) DF2_pr (우울증 현재 유병 여부): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (30) DF2_pt (우울증 치료): 0.없음, 1.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (31) DL1_dg (아토피피부염 의사진단 여부): 0.없음, 1.있음,  9.모름, 무응답
#### (32) DJ8_dg (알레르기비염 의사진단 여부): 0.없음, 1.있음,  9.모름, 무응답
#### (33) LQ_1EQL (EuroQoL: 운동능력): 1.나는 걷는데 지장이 없음, 2.나는 걷는데 다소 지장이 있음, 3.나는 종일 누워 있어야 함, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (34) LQ_2EQL (EuroQoL: 자기관리): 1.나는 목욕을 하거나 옷을 입는데 지장이 없음, 2.나는 혼자 목욕을 하거나 옷을 입는데 다소 지장이 있음, 3.나는 혼자 목욕을 하거나 옷을 입을 수 없음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (35) LQ_3EQL (EuroQoL: 일상활동): 1.나는 일상활동을 하는데 지장 없음, 2.나는 일상활동을 하는데 다소 지장 있음, 3.나는 일상활동을 할 수 없음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (36) LQ_4EQL (EuroQoL: 통증/불편): 1.나는 통증/불편감이 없음, 2.나는 다소 통증/불편감이 있음, 3.나는 매우 심한 통증/불편감 있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (37) LQ_5EQL (EuroQoL: 불안/우울): 1.나는 불안하거나 우울하지 않음, 2.나는 다소 불안하거나 우울함, 3.나는 매우 심하게 불안하거나 우울함, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (38) EQ5D (EQ-5D index; 0~1 사이로 1 에 가까울수록 삶의 질이 높음을 의미함): 

$\text{EQ-5D Index}=1-[0.05 + 0.096*(1:\text{LQ_1EQL=2}, 0:\text{LQ_1EQL≠2}) + 0.418*(1:\text{LQ_1EQL=3}, 0:\text{LQ_1EQL≠3}) + 0.046*(1:\text{LQ_2EQL=2}, 0:\text{LQ_2EQL≠2}) + 0.136*(1:\text{LQ_2EQL=3}, 0:\text{LQ_2EQL≠3}) + 0.051*(1:\text{LQ_3EQL=2}, 0:\text{LQ_3EQL≠2}) + 0.208*(1:\text{LQ_3EQL=3}, 0:\text{LQ_3EQL≠3}) + 0.037*(\text{1:LQ_4EQL=2}, 0:\text{LQ_4EQL≠2}) + 0.151*(1:\text{LQ_4EQL=3}, 0:\text{LQ_4EQL≠3}) + 0.043*(1:\text{LQ_5EQL=2}, 0:\text{LQ_5EQL≠2}) + 0.158*(1:\text{LQ_5EQL=3}, 0:\text{LQ_5EQL≠3}) + 0.05*(1:\text{LQ_1EQL~LQ_5EQL=3}, 0:\text{LQ_1EQL~LQ_5EQL≠3})]$

#### (39) BO1_1 (1년간 체중 변화 여부): 1.변화 없음, 2.체중 감소, 3.체중 증가, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (40) BD1 (평생음주경험): 1.술을 마셔 본 적 없음, 2.있음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (41) mh_PHQ_S (PHQ-9 점수): 우울증선별도구 9항목(각 0~3점) 점수 합
#### (42) BP1 (평소 스트레스 인지 정도): 1.대단히 많이 느낀다, 2.많이 느끼는 편이다, 3.조금 느끼는 편이다, 4.거의 느끼지 않는다, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (43) BS1_1 (평생 일반담배(궐련) 흡연 여부): 1.5갑(100개비) 미만, 2.5갑(100개비) 이상, 3.피운 적 없음, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (44) sm_presnt (현재흡연율): 0.과거 흡연, 비흡연, 1.현재흡연
#### (45) BE3_85 (중강도 신체활동 여부: 여가): 1.예, 2.아니오, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (46) BE3_86 (중강도 신체활동 일수: 여가): xx일 (일주일 동안), 8.비해당(청소년, 소아), 9.모름, 무응답
#### (47) BE3_75 (고강도 신체활동 여부: 여가): 1.예, 2.아니오, 8.비해당(청소년, 소아), 9.모름, 무응답
#### (48) BE3_76 (고강도 신체활동 일수: 여가): xx일 (일주일 동안), 8.비해당(청소년, 소아), 9.모름, 무응답
#### (49) HE_BMI (체질량지수): xx kg/m2
#### (50) HE_glu (공복혈당): xx mg/dL
#### (51) HE_chol (총콜레스테롤): xx mg/dL
#### (52) HE_TG (중성지방): xx mg/dL

결측치 제거 : 비해당 및 무응답 값

In [12]:
df1a=df1a.reset_index(drop=True)

df2a=df2a.reset_index(drop=True)

df3a=df3a.reset_index(drop=True)

df4a=df4a.reset_index(drop=True)

빈 리스트로 무응답 index append 및 제거

In [13]:
idx1 = df1a[df1a['genertn']==9].index
idx2= df1a[df1a['allownc']==99].index
idx3= df1a[df1a['marri_1']==9].index
idx7= df1a[df1a['DI1_dg']==9].index
idx8= df1a[df1a['DI2_dg']==9].index
idx9= df1a[df1a['DI3_dg']==9].index
idx10= df1a[df1a['DI5_dg']==9].index
idx11= df1a[df1a['DI6_dg']==9].index
idx12= df1a[df1a['DM2_dg']==9].index
idx13= df1a[df1a['DM3_dg']==9].index
idx14= df1a[df1a['DJ2_dg']==9].index
idx15= df1a[df1a['DE2_dg']==9].index
idx16= df1a[df1a['DE1_dg']==9].index
idx17= df1a[df1a['DC1_dg']==9].index
idx18= df1a[df1a['DC2_dg']==9].index
idx19= df1a[df1a['DC3_dg']==9].index
idx20= df1a[df1a['DC4_dg']==9].index
idx21= df1a[df1a['DC5_dg']==9].index
idx22= df1a[df1a['DC6_dg']==9].index
idx23= df1a[df1a['DC7_dg']==9].index
idx24= df1a[df1a['DF2_dg']==9].index
idx25= df1a[df1a['DF2_pr']==9].index
idx26= df1a[df1a['DF2_pt']==9].index
idx27= df1a[df1a['DL1_dg']==9].index
idx28= df1a[df1a['DJ8_dg']==9].index
idx29= df1a[df1a['LQ_1EQL']==9].index
idx30= df1a[df1a['LQ_2EQL']==9].index
idx31= df1a[df1a['LQ_3EQL']==9].index
idx32= df1a[df1a['LQ_4EQL']==9].index
idx33= df1a[df1a['LQ_5EQL']==9].index
idx34= df1a[df1a['BO1_1']==9].index
idx35= df1a[df1a['BD1']==9].index
idx36= df1a[df1a['BP1']==9].index
idx37= df1a[df1a['BP6_10']==9].index
idx38= df1a[df1a['BP6_2']==9].index
idx39= df1a[df1a['BP6_31']==9].index
idx40= df1a[df1a['BS1_1']==9].index
idx41= df1a[df1a['BE3_85']==9].index
idx42= df1a[df1a['BE3_86']==9].index
idx43= df1a[df1a['BE3_75']==9].index
idx44= df1a[df1a['BE3_76']==9].index

idx45= df1a[df1a['DI1_dg']==8].index
idx46= df1a[df1a['DI2_dg']==8].index
idx47= df1a[df1a['DI3_dg']==8].index
idx48= df1a[df1a['DI5_dg']==8].index
idx49= df1a[df1a['DI6_dg']==8].index
idx50= df1a[df1a['DM2_dg']==8].index
idx51= df1a[df1a['DM3_dg']==8].index
idx52= df1a[df1a['DJ2_dg']==8].index
idx53= df1a[df1a['DE2_dg']==8].index
idx54= df1a[df1a['DE1_dg']==8].index
idx55= df1a[df1a['DC1_dg']==8].index
idx56= df1a[df1a['DC2_dg']==8].index
idx57= df1a[df1a['DC3_dg']==8].index
idx58= df1a[df1a['DC4_dg']==8].index
idx59= df1a[df1a['DC5_dg']==8].index
idx60= df1a[df1a['DC6_dg']==8].index
idx61= df1a[df1a['DC7_dg']==8].index
idx62= df1a[df1a['DF2_dg']==8].index
idx67= df1a[df1a['LQ_1EQL']==8].index
idx68= df1a[df1a['LQ_2EQL']==8].index
idx69= df1a[df1a['LQ_3EQL']==8].index
idx70= df1a[df1a['LQ_4EQL']==8].index
idx71= df1a[df1a['LQ_5EQL']==8].index
idx71= df1a[df1a['BO1_1']==8].index
idx72= df1a[df1a['BD1']==8].index
idx73= df1a[df1a['BP1']==8].index
idx77= df1a[df1a['BS1_1']==8].index
idx78= df1a[df1a['BE3_85']==8].index
idx80= df1a[df1a['BE3_75']==8].index

In [14]:
idx=df1a[df1a['genertn']==''].index
idx=idx.append(idx1)
idx=idx.append(idx2)
idx=idx.append(idx3)
idx=idx.append(idx7)
idx=idx.append(idx8)
idx=idx.append(idx9)
idx=idx.append(idx10)
idx=idx.append(idx11)
idx=idx.append(idx12)
idx=idx.append(idx13)
idx=idx.append(idx14)
idx=idx.append(idx15)
idx=idx.append(idx16)
idx=idx.append(idx17)
idx=idx.append(idx18)
idx=idx.append(idx19)
idx=idx.append(idx20)
idx=idx.append(idx21)
idx=idx.append(idx22)
idx=idx.append(idx23)
idx=idx.append(idx24)
idx=idx.append(idx25)
idx=idx.append(idx26)
idx=idx.append(idx27)
idx=idx.append(idx28)
idx=idx.append(idx29)
idx=idx.append(idx30)
idx=idx.append(idx31)
idx=idx.append(idx32)
idx=idx.append(idx33)
idx=idx.append(idx34)
idx=idx.append(idx35)
idx=idx.append(idx36)
idx=idx.append(idx37)
idx=idx.append(idx38)
idx=idx.append(idx39)
idx=idx.append(idx40)
idx=idx.append(idx41)
idx=idx.append(idx42)
idx=idx.append(idx43)
idx=idx.append(idx44)

idx=idx.append(idx45)
idx=idx.append(idx46)
idx=idx.append(idx47)
idx=idx.append(idx48)
idx=idx.append(idx49)
idx=idx.append(idx50)
idx=idx.append(idx51)
idx=idx.append(idx52)
idx=idx.append(idx53)
idx=idx.append(idx54)
idx=idx.append(idx55)
idx=idx.append(idx56)
idx=idx.append(idx57)
idx=idx.append(idx58)
idx=idx.append(idx59)
idx=idx.append(idx60)
idx=idx.append(idx61)
idx=idx.append(idx62)
idx=idx.append(idx67)
idx=idx.append(idx68)
idx=idx.append(idx69)
idx=idx.append(idx70)
idx=idx.append(idx71)
idx=idx.append(idx72)
idx=idx.append(idx73)
idx=idx.append(idx77)
idx=idx.append(idx78)
idx=idx.append(idx80)

idx_list=set(idx)
idx_list=list(idx_list)
idx_pd=pd.Index(idx_list)

df1a=df1a.drop(index=idx_pd,axis=0)

In [15]:
idx1 = df2a[df2a['genertn']==9].index
idx2= df2a[df2a['allownc']==99].index
idx3= df2a[df2a['marri_1']==9].index
idx7= df2a[df2a['DI1_dg']==9].index
idx8= df2a[df2a['DI2_dg']==9].index
idx9= df2a[df2a['DI3_dg']==9].index
idx10= df2a[df2a['DI5_dg']==9].index
idx11= df2a[df2a['DI6_dg']==9].index
idx12= df2a[df2a['DM2_dg']==9].index
idx13= df2a[df2a['DM3_dg']==9].index
idx14= df2a[df2a['DJ2_dg']==9].index
idx15= df2a[df2a['DE2_dg']==9].index
idx16= df2a[df2a['DE1_dg']==9].index
idx17= df2a[df2a['DC1_dg']==9].index
idx18= df2a[df2a['DC2_dg']==9].index
idx19= df2a[df2a['DC3_dg']==9].index
idx20= df2a[df2a['DC4_dg']==9].index
idx21= df2a[df2a['DC5_dg']==9].index
idx22= df2a[df2a['DC6_dg']==9].index
idx23= df2a[df2a['DC7_dg']==9].index
idx24= df2a[df2a['DF2_dg']==9].index
idx25= df2a[df2a['DF2_pr']==9].index
idx26= df2a[df2a['DF2_pt']==9].index
idx27= df2a[df2a['DL1_dg']==9].index
idx28= df2a[df2a['DJ8_dg']==9].index
idx29= df2a[df2a['LQ_1EQL']==9].index
idx30= df2a[df2a['LQ_2EQL']==9].index
idx31= df2a[df2a['LQ_3EQL']==9].index
idx32= df2a[df2a['LQ_4EQL']==9].index
idx33= df2a[df2a['LQ_5EQL']==9].index
idx34= df2a[df2a['BO1_1']==9].index
idx35= df2a[df2a['BD1']==9].index
idx36= df2a[df2a['BP1']==9].index
idx37= df2a[df2a['BP6_10']==9].index
idx38= df2a[df2a['BP6_2']==9].index
idx39= df2a[df2a['BP6_31']==9].index
idx40= df2a[df2a['BS1_1']==9].index
idx41= df2a[df2a['BE3_85']==9].index
idx42= df2a[df2a['BE3_86']==9].index
idx43= df2a[df2a['BE3_75']==9].index
idx44= df2a[df2a['BE3_76']==9].index

idx45= df2a[df2a['DI1_dg']==8].index
idx46= df2a[df2a['DI2_dg']==8].index
idx47= df2a[df2a['DI3_dg']==8].index
idx48= df2a[df2a['DI5_dg']==8].index
idx49= df2a[df2a['DI6_dg']==8].index
idx50= df2a[df2a['DM2_dg']==8].index
idx51= df2a[df2a['DM3_dg']==8].index
idx52= df2a[df2a['DJ2_dg']==8].index
idx53= df2a[df2a['DE2_dg']==8].index
idx54= df2a[df2a['DE1_dg']==8].index
idx55= df2a[df2a['DC1_dg']==8].index
idx56= df2a[df2a['DC2_dg']==8].index
idx57= df2a[df2a['DC3_dg']==8].index
idx58= df2a[df2a['DC4_dg']==8].index
idx59= df2a[df2a['DC5_dg']==8].index
idx60= df2a[df2a['DC6_dg']==8].index
idx61= df2a[df2a['DC7_dg']==8].index
idx62= df2a[df2a['DF2_dg']==8].index
idx67= df2a[df2a['LQ_1EQL']==8].index
idx68= df2a[df2a['LQ_2EQL']==8].index
idx69= df2a[df2a['LQ_3EQL']==8].index
idx70= df2a[df2a['LQ_4EQL']==8].index
idx71= df2a[df2a['LQ_5EQL']==8].index
idx71= df2a[df2a['BO1_1']==8].index
idx72= df2a[df2a['BD1']==8].index
idx73= df2a[df2a['BP1']==8].index
idx77= df2a[df2a['BS1_1']==8].index
idx78= df2a[df2a['BE3_85']==8].index
idx80= df2a[df2a['BE3_75']==8].index

In [16]:
idx=df2a[df2a['genertn']==''].index
idx=idx.append(idx1)
idx=idx.append(idx2)
idx=idx.append(idx3)
idx=idx.append(idx7)
idx=idx.append(idx8)
idx=idx.append(idx9)
idx=idx.append(idx10)
idx=idx.append(idx11)
idx=idx.append(idx12)
idx=idx.append(idx13)
idx=idx.append(idx14)
idx=idx.append(idx15)
idx=idx.append(idx16)
idx=idx.append(idx17)
idx=idx.append(idx18)
idx=idx.append(idx19)
idx=idx.append(idx20)
idx=idx.append(idx21)
idx=idx.append(idx22)
idx=idx.append(idx23)
idx=idx.append(idx24)
idx=idx.append(idx25)
idx=idx.append(idx26)
idx=idx.append(idx27)
idx=idx.append(idx28)
idx=idx.append(idx29)
idx=idx.append(idx30)
idx=idx.append(idx31)
idx=idx.append(idx32)
idx=idx.append(idx33)
idx=idx.append(idx34)
idx=idx.append(idx35)
idx=idx.append(idx36)
idx=idx.append(idx37)
idx=idx.append(idx38)
idx=idx.append(idx39)
idx=idx.append(idx40)
idx=idx.append(idx41)
idx=idx.append(idx42)
idx=idx.append(idx43)
idx=idx.append(idx44)

idx=idx.append(idx45)
idx=idx.append(idx46)
idx=idx.append(idx47)
idx=idx.append(idx48)
idx=idx.append(idx49)
idx=idx.append(idx50)
idx=idx.append(idx51)
idx=idx.append(idx52)
idx=idx.append(idx53)
idx=idx.append(idx54)
idx=idx.append(idx55)
idx=idx.append(idx56)
idx=idx.append(idx57)
idx=idx.append(idx58)
idx=idx.append(idx59)
idx=idx.append(idx60)
idx=idx.append(idx61)
idx=idx.append(idx62)
idx=idx.append(idx67)
idx=idx.append(idx68)
idx=idx.append(idx69)
idx=idx.append(idx70)
idx=idx.append(idx71)
idx=idx.append(idx72)
idx=idx.append(idx73)
idx=idx.append(idx77)
idx=idx.append(idx78)
idx=idx.append(idx80)

idx_list=set(idx)
idx_list=list(idx_list)
idx_pd=pd.Index(idx_list)

df2a=df2a.drop(index=idx_pd,axis=0)

In [17]:
idx1 = df3a[df3a['genertn']==9].index
idx2= df3a[df3a['allownc']==99].index
idx3= df3a[df3a['marri_1']==9].index
idx7= df3a[df3a['DI1_dg']==9].index
idx8= df3a[df3a['DI2_dg']==9].index
idx9= df3a[df3a['DI3_dg']==9].index
idx10= df3a[df3a['DI5_dg']==9].index
idx11= df3a[df3a['DI6_dg']==9].index
idx12= df3a[df3a['DM2_dg']==9].index
idx13= df3a[df3a['DM3_dg']==9].index
idx14= df3a[df3a['DJ2_dg']==9].index
idx15= df3a[df3a['DE2_dg']==9].index
idx16= df3a[df3a['DE1_dg']==9].index
idx17= df3a[df3a['DC1_dg']==9].index
idx18= df3a[df3a['DC2_dg']==9].index
idx19= df3a[df3a['DC3_dg']==9].index
idx20= df3a[df3a['DC4_dg']==9].index
idx21= df3a[df3a['DC5_dg']==9].index
idx22= df3a[df3a['DC6_dg']==9].index
idx23= df3a[df3a['DC7_dg']==9].index
idx24= df3a[df3a['DF2_dg']==9].index
idx25= df3a[df3a['DF2_pr']==9].index
idx26= df3a[df3a['DF2_pt']==9].index
idx27= df3a[df3a['DL1_dg']==9].index
idx28= df3a[df3a['DJ8_dg']==9].index
idx29= df3a[df3a['LQ_1EQL']==9].index
idx30= df3a[df3a['LQ_2EQL']==9].index
idx31= df3a[df3a['LQ_3EQL']==9].index
idx32= df3a[df3a['LQ_4EQL']==9].index
idx33= df3a[df3a['LQ_5EQL']==9].index
idx34= df3a[df3a['BO1_1']==9].index
idx35= df3a[df3a['BD1']==9].index
idx36= df3a[df3a['BP1']==9].index
idx37= df3a[df3a['BP6_10']==9].index
idx38= df3a[df3a['BP6_2']==9].index
idx39= df3a[df3a['BP6_31']==9].index
idx40= df3a[df3a['BS1_1']==9].index
idx41= df3a[df3a['BE3_85']==9].index
idx42= df3a[df3a['BE3_86']==9].index
idx43= df3a[df3a['BE3_75']==9].index
idx44= df3a[df3a['BE3_76']==9].index

idx45= df3a[df3a['DI1_dg']==8].index
idx46= df3a[df3a['DI2_dg']==8].index
idx47= df3a[df3a['DI3_dg']==8].index
idx48= df3a[df3a['DI5_dg']==8].index
idx49= df3a[df3a['DI6_dg']==8].index
idx50= df3a[df3a['DM2_dg']==8].index
idx51= df3a[df3a['DM3_dg']==8].index
idx52= df3a[df3a['DJ2_dg']==8].index
idx53= df3a[df3a['DE2_dg']==8].index
idx54= df3a[df3a['DE1_dg']==8].index
idx55= df3a[df3a['DC1_dg']==8].index
idx56= df3a[df3a['DC2_dg']==8].index
idx57= df3a[df3a['DC3_dg']==8].index
idx58= df3a[df3a['DC4_dg']==8].index
idx59= df3a[df3a['DC5_dg']==8].index
idx60= df3a[df3a['DC6_dg']==8].index
idx61= df3a[df3a['DC7_dg']==8].index
idx62= df3a[df3a['DF2_dg']==8].index
idx67= df3a[df3a['LQ_1EQL']==8].index
idx68= df3a[df3a['LQ_2EQL']==8].index
idx69= df3a[df3a['LQ_3EQL']==8].index
idx70= df3a[df3a['LQ_4EQL']==8].index
idx71= df3a[df3a['LQ_5EQL']==8].index
idx71= df3a[df3a['BO1_1']==8].index
idx72= df3a[df3a['BD1']==8].index
idx73= df3a[df3a['BP1']==8].index
idx77= df3a[df3a['BS1_1']==8].index
idx78= df3a[df3a['BE3_85']==8].index
idx80= df3a[df3a['BE3_75']==8].index

In [18]:
idx=df3a[df3a['genertn']==''].index
idx=idx.append(idx1)
idx=idx.append(idx2)
idx=idx.append(idx3)
idx=idx.append(idx7)
idx=idx.append(idx8)
idx=idx.append(idx9)
idx=idx.append(idx10)
idx=idx.append(idx11)
idx=idx.append(idx12)
idx=idx.append(idx13)
idx=idx.append(idx14)
idx=idx.append(idx15)
idx=idx.append(idx16)
idx=idx.append(idx17)
idx=idx.append(idx18)
idx=idx.append(idx19)
idx=idx.append(idx20)
idx=idx.append(idx21)
idx=idx.append(idx22)
idx=idx.append(idx23)
idx=idx.append(idx24)
idx=idx.append(idx25)
idx=idx.append(idx26)
idx=idx.append(idx27)
idx=idx.append(idx28)
idx=idx.append(idx29)
idx=idx.append(idx30)
idx=idx.append(idx31)
idx=idx.append(idx32)
idx=idx.append(idx33)
idx=idx.append(idx34)
idx=idx.append(idx35)
idx=idx.append(idx36)
idx=idx.append(idx37)
idx=idx.append(idx38)
idx=idx.append(idx39)
idx=idx.append(idx40)
idx=idx.append(idx41)
idx=idx.append(idx42)
idx=idx.append(idx43)
idx=idx.append(idx44)

idx=idx.append(idx45)
idx=idx.append(idx46)
idx=idx.append(idx47)
idx=idx.append(idx48)
idx=idx.append(idx49)
idx=idx.append(idx50)
idx=idx.append(idx51)
idx=idx.append(idx52)
idx=idx.append(idx53)
idx=idx.append(idx54)
idx=idx.append(idx55)
idx=idx.append(idx56)
idx=idx.append(idx57)
idx=idx.append(idx58)
idx=idx.append(idx59)
idx=idx.append(idx60)
idx=idx.append(idx61)
idx=idx.append(idx62)
idx=idx.append(idx67)
idx=idx.append(idx68)
idx=idx.append(idx69)
idx=idx.append(idx70)
idx=idx.append(idx71)
idx=idx.append(idx72)
idx=idx.append(idx73)
idx=idx.append(idx77)
idx=idx.append(idx78)
idx=idx.append(idx80)

idx_list=set(idx)
idx_list=list(idx_list)
idx_pd=pd.Index(idx_list)

df3a=df3a.drop(index=idx_pd,axis=0)

In [19]:
idx1 = df4a[df4a['genertn']==9].index
idx2= df4a[df4a['allownc']==99].index
idx3= df4a[df4a['marri_1']==9].index
idx7= df4a[df4a['DI1_dg']==9].index
idx8= df4a[df4a['DI2_dg']==9].index
idx9= df4a[df4a['DI3_dg']==9].index
idx10= df4a[df4a['DI5_dg']==9].index
idx11= df4a[df4a['DI6_dg']==9].index
idx12= df4a[df4a['DM2_dg']==9].index
idx13= df4a[df4a['DM3_dg']==9].index
idx14= df4a[df4a['DJ2_dg']==9].index
idx15= df4a[df4a['DE2_dg']==9].index
idx16= df4a[df4a['DE1_dg']==9].index
idx17= df4a[df4a['DC1_dg']==9].index
idx18= df4a[df4a['DC2_dg']==9].index
idx19= df4a[df4a['DC3_dg']==9].index
idx20= df4a[df4a['DC4_dg']==9].index
idx21= df4a[df4a['DC5_dg']==9].index
idx22= df4a[df4a['DC6_dg']==9].index
idx23= df4a[df4a['DC7_dg']==9].index
idx24= df4a[df4a['DF2_dg']==9].index
idx25= df4a[df4a['DF2_pr']==9].index
idx26= df4a[df4a['DF2_pt']==9].index
idx27= df4a[df4a['DL1_dg']==9].index
idx28= df4a[df4a['DJ8_dg']==9].index
idx29= df4a[df4a['LQ_1EQL']==9].index
idx30= df4a[df4a['LQ_2EQL']==9].index
idx31= df4a[df4a['LQ_3EQL']==9].index
idx32= df4a[df4a['LQ_4EQL']==9].index
idx33= df4a[df4a['LQ_5EQL']==9].index
idx34= df4a[df4a['BO1_1']==9].index
idx35= df4a[df4a['BD1']==9].index
idx36= df4a[df4a['BP1']==9].index
idx37= df4a[df4a['BP6_10']==9].index
idx38= df4a[df4a['BP6_2']==9].index
idx39= df4a[df4a['BP6_31']==9].index
idx40= df4a[df4a['BS1_1']==9].index
idx41= df4a[df4a['BE3_85']==9].index
idx42= df4a[df4a['BE3_86']==9].index
idx43= df4a[df4a['BE3_75']==9].index
idx44= df4a[df4a['BE3_76']==9].index

idx45= df4a[df4a['DI1_dg']==8].index
idx46= df4a[df4a['DI2_dg']==8].index
idx47= df4a[df4a['DI3_dg']==8].index
idx48= df4a[df4a['DI5_dg']==8].index
idx49= df4a[df4a['DI6_dg']==8].index
idx50= df4a[df4a['DM2_dg']==8].index
idx51= df4a[df4a['DM3_dg']==8].index
idx52= df4a[df4a['DJ2_dg']==8].index
idx53= df4a[df4a['DE2_dg']==8].index
idx54= df4a[df4a['DE1_dg']==8].index
idx55= df4a[df4a['DC1_dg']==8].index
idx56= df4a[df4a['DC2_dg']==8].index
idx57= df4a[df4a['DC3_dg']==8].index
idx58= df4a[df4a['DC4_dg']==8].index
idx59= df4a[df4a['DC5_dg']==8].index
idx60= df4a[df4a['DC6_dg']==8].index
idx61= df4a[df4a['DC7_dg']==8].index
idx62= df4a[df4a['DF2_dg']==8].index
idx67= df4a[df4a['LQ_1EQL']==8].index
idx68= df4a[df4a['LQ_2EQL']==8].index
idx69= df4a[df4a['LQ_3EQL']==8].index
idx70= df4a[df4a['LQ_4EQL']==8].index
idx71= df4a[df4a['LQ_5EQL']==8].index
idx71= df4a[df4a['BO1_1']==8].index
idx72= df4a[df4a['BD1']==8].index
idx73= df4a[df4a['BP1']==8].index
idx77= df4a[df4a['BS1_1']==8].index
idx78= df4a[df4a['BE3_85']==8].index
idx80= df4a[df4a['BE3_75']==8].index

In [20]:
idx=df4a[df4a['genertn']==''].index
idx=idx.append(idx1)
idx=idx.append(idx2)
idx=idx.append(idx3)
idx=idx.append(idx7)
idx=idx.append(idx8)
idx=idx.append(idx9)
idx=idx.append(idx10)
idx=idx.append(idx11)
idx=idx.append(idx12)
idx=idx.append(idx13)
idx=idx.append(idx14)
idx=idx.append(idx15)
idx=idx.append(idx16)
idx=idx.append(idx17)
idx=idx.append(idx18)
idx=idx.append(idx19)
idx=idx.append(idx20)
idx=idx.append(idx21)
idx=idx.append(idx22)
idx=idx.append(idx23)
idx=idx.append(idx24)
idx=idx.append(idx25)
idx=idx.append(idx26)
idx=idx.append(idx27)
idx=idx.append(idx28)
idx=idx.append(idx29)
idx=idx.append(idx30)
idx=idx.append(idx31)
idx=idx.append(idx32)
idx=idx.append(idx33)
idx=idx.append(idx34)
idx=idx.append(idx35)
idx=idx.append(idx36)
idx=idx.append(idx37)
idx=idx.append(idx38)
idx=idx.append(idx39)
idx=idx.append(idx40)
idx=idx.append(idx41)
idx=idx.append(idx42)
idx=idx.append(idx43)
idx=idx.append(idx44)

idx=idx.append(idx45)
idx=idx.append(idx46)
idx=idx.append(idx47)
idx=idx.append(idx48)
idx=idx.append(idx49)
idx=idx.append(idx50)
idx=idx.append(idx51)
idx=idx.append(idx52)
idx=idx.append(idx53)
idx=idx.append(idx54)
idx=idx.append(idx55)
idx=idx.append(idx56)
idx=idx.append(idx57)
idx=idx.append(idx58)
idx=idx.append(idx59)
idx=idx.append(idx60)
idx=idx.append(idx61)
idx=idx.append(idx62)
idx=idx.append(idx67)
idx=idx.append(idx68)
idx=idx.append(idx69)
idx=idx.append(idx70)
idx=idx.append(idx71)
idx=idx.append(idx72)
idx=idx.append(idx73)
idx=idx.append(idx77)
idx=idx.append(idx78)
idx=idx.append(idx80)

idx_list=set(idx)
idx_list=list(idx_list)
idx_pd=pd.Index(idx_list)

df4a=df4a.drop(index=idx_pd,axis=0)

변수 추가 전처리
 - 우울증 유병 및 치료 비해당 시 없음으로 표기
 - 중강도 및 고강도 신체활동에서 비해당 시 0일로 표기
 - 이후 0일, 1-2일, 3-4일, 5일<=으로 카테고리화

In [61]:
df1a['DF2_pr']=df1a['DF2_pr'].apply(lambda x: 0 if x==8 else x)
df1a['DF2_pt']=df1a['DF2_pt'].apply(lambda x: 0 if x==8 else x)
df1a['BE3_86']=df1a['BE3_86'].apply(lambda x: 0 if x==8 else x)
df1a['BE3_76']=df1a['BE3_76'].apply(lambda x: 0 if x==8 else x)

df2a['DF2_pr']=df2a['DF2_pr'].apply(lambda x: 0 if x==8 else x)
df2a['DF2_pt']=df2a['DF2_pt'].apply(lambda x: 0 if x==8 else x)
df2a['BE3_86']=df2a['BE3_86'].apply(lambda x: 0 if x==8 else x)
df2a['BE3_76']=df2a['BE3_76'].apply(lambda x: 0 if x==8 else x)

df3a['DF2_pr']=df3a['DF2_pr'].apply(lambda x: 0 if x==8 else x)
df3a['DF2_pt']=df3a['DF2_pt'].apply(lambda x: 0 if x==8 else x)
df3a['BE3_86']=df3a['BE3_86'].apply(lambda x: 0 if x==8 else x)
df3a['BE3_76']=df3a['BE3_76'].apply(lambda x: 0 if x==8 else x)

df4a['DF2_pr']=df4a['DF2_pr'].apply(lambda x: 0 if x==8 else x)
df4a['DF2_pt']=df4a['DF2_pt'].apply(lambda x: 0 if x==8 else x)
df4a['BE3_86']=df4a['BE3_86'].apply(lambda x: 0 if x==8 else x)
df4a['BE3_76']=df4a['BE3_76'].apply(lambda x: 0 if x==8 else x)

In [62]:
def exercise(x):
    if x ==0:
       return 0
    elif 0< x < 3:
       return 1
    elif 3<= x < 5:
       return 2
    else:
       return 3

df1a['mpa_g']=df1a['BE3_86'].apply(exercise)
df1a['vpa_g']=df1a['BE3_76'].apply(exercise)

df2a['mpa_g']=df2a['BE3_86'].apply(exercise)
df2a['vpa_g']=df2a['BE3_76'].apply(exercise)

df3a['mpa_g']=df3a['BE3_86'].apply(exercise)
df3a['vpa_g']=df3a['BE3_76'].apply(exercise)

df4a['mpa_g']=df4a['BE3_86'].apply(exercise)
df4a['vpa_g']=df4a['BE3_76'].apply(exercise)

In [70]:
train = pd.concat([df1a, df2a, df3a])
test = df4a

In [71]:
test.to_csv('train.csv')
train.to_csv('test.csv')